In [49]:
import torch
import torchvision.models as models
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn import svm
import ssl
import matplotlib.pyplot as plt
import os
import copy
import csv
ssl._create_default_https_context = ssl._create_unverified_context

In [50]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_dataset = datasets.ImageFolder('../Data/train', transform=transform)
test_dataset = datasets.ImageFolder('../Data/test', transform=transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
# dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}

In [54]:
resnet18 = models.resnet18(ResNet18_Weights.DEFAULT)
model_test = list(resnet18.children())[:-1]
model = nn.Sequential(*model_test).to(device)
model.eval()
model.to(device)
isInit = True
with torch.no_grad():
    for image, label in train_dataloader:
        image.to(device)
        label.to(device)
        output = model(image)
        output = output[0].view(-1, 1)
        label = label.view(-1, 1)
        test = torch.cat((output, label))
        test = test.view(1, -1)
        bb = test.numpy()
        cc = pd.DataFrame(bb)
        cc.to_csv('../Data/train_features.csv', mode='a', header=isInit)
        isInit = False

print('finished')

/Users/shanw25/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


finished
